In [1]:
#importar as biblioteca
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import plotly.offline as po
import tkinter as tk
from tkinter import messagebox, filedialog
import os 

In [2]:
cor1 = ["#000000", "#808080", "#7B68EE", "#4B0082", "#BA55D3", "#8B008B", "#FF00FF", "#6A5ACD", "#00008B", "#00BFFF", "#ADD8E6",
       "#FF69B4", "#DC143C", "#00FFFF", "#8B0000", "#E9967A", "#FF0000", "#D2B48C", "#FF8C00", "#FFFF00", "#2F4F4F", "#00FF7F"]
cor = ["#9ACD32", "#808000", "#8B4513", "#D2691E", "#D8BFD8", "#AE5687"]
cor.extend(cor1) # Criação de cores em tuplas com extenção das tuplas

In [3]:
sns.color_palette('BrBG', 29)
sns.set(context='notebook', style='darkgrid', font='sans-serif', font_scale=1.35)
#table = ff.create_table(df_glpi)
#po.iplot(table, filename='jupyter/table1')
#df_glpi.head()

In [ ]:


class Tela:
    def __init__(self, master=None):
        
        self.nossatela = master
        self.lbl = tk.Label(self.nossatela, text="GLPI", font=('helvetica', 12, 'bold'), bg='#dde')
        self.lbl.pack(pady=18)
        
        botaoAbrir = tk.Button(janelaRaiz, text="Abrir Arquivo ", command = self.abrirArquivo,  bg='green', fg='white', font=('helvetica', 12, 'bold'))
        botaoAbrir.place(x=200, y=60)#pack()
        
        self.gera_button = tk.Button(master, text="Gerar Relatório", font=('18'), command=self.dat)
        self.gera_button.place(x=200, y=120)#pack()
        
        df_glpi = pd.DataFrame()
        
    def abrirArquivo(self):
        global df_glpi
        
        arquivo = filedialog.askopenfilename(initialdir = '/Desktop',defaultextension='.csv', filetypes=[("CSV files","*.csv")])
        df_glpi = pd.read_csv(arquivo, sep=';', index_col = 'ID') #Carregar o csv
        df_glpi = df_glpi.join(df_glpi['Atribuído para - Técnico'].str.rsplit('\n', n=3, expand = True).rename({0:'Técnico - Principal', 1:'Técnico - Auxiliar', 2:'Técnico - Auxiliar (2)', 3:'Técnico - Auxiliar (3)'}, axis=1))
        df_glpi[['Técnico - Principal','Técnico - Auxiliar','Técnico - Auxiliar (2)']] = df_glpi[['Técnico - Principal','Técnico - Auxiliar','Técnico - Auxiliar (2)']].replace(to_replace=r'\r', value='', regex=True)
        df_glpi['Categoria'].replace({'MANUTENÇÃO':'MANUTENÇÃO > Manutenção', 'REDE':'REDE > Rede', 'SigUEMA':'SIGUEMA > SigUEMA'}, inplace=True)
        df_glpi = df_glpi.join(df_glpi['Categoria'].str.rsplit(' > ', n=2, expand = True).rename({0:'Categoria1', 1:'Area', 2:'Sub'}, axis=1))
        df_glpi = df_glpi.join(df_glpi['Localização'].str.rsplit(' > ', n=1, expand = True).rename({0:'Predio', 1:'Curso'}, axis=1))
        df_glpi = df_glpi.join(df_glpi['Requerente - Autor'].str.split(n=1, expand = True).rename({0:'Cpf', 1:'Helpdesk'}, axis=1))
        df_glpi = df_glpi.join(df_glpi['Última atualização'].astype(str).str.rsplit("-", n=1, expand = True).rename({0:'Data', 1:'dia'}, axis=1))
        df_glpi.drop(columns=['Atribuído para - Técnico', 'Categoria', 'Categoria1', 'Sub', 'Localização', 'Curso', 'Requerente - Autor', 'Cpf', 'dia', 'Última atualização', 'Unnamed: 13'], inplace=True)

        df_glpi['Predio'] = df_glpi['Predio'].map({'ASSUEMA - Associação dos Servidores da UEMA' : 'ASSUEMA'}, na_action=None)
        tecn_Aux2 = df_glpi['Técnico - Auxiliar (2)']
        tecn_Aux3 = df_glpi['Técnico - Auxiliar (3)']
        tecn_Aux = df_glpi['Técnico - Auxiliar']
        tecn_Prin = df_glpi['Técnico - Principal']

        df_tecn = pd.DataFrame(pd.concat([tecn_Prin, tecn_Aux.dropna(), tecn_Aux2.dropna(), 
                                          tecn_Aux3.dropna()])).rename({0:'Tecnicos'}, axis=1)
        df_final = pd.merge(df_tecn, df_glpi,how='left', on='ID')
        df_final.drop(columns=['Técnico - Principal', 'Técnico - Auxiliar', 'Técnico - Auxiliar (2)', 
                               'Técnico - Auxiliar (3)'], inplace=True)


    def dat(self):
        self.datas = tk.Entry(width=25, bg='white')
        self.datas.place(x=20, y=190)
        self.Info1 = tk.Label(font=('Arial', '11', 'bold'), fg='#009', bg='#dde', text='Qual ano e mês (ex: 2020-01): ')
        self.Info1.place(x=20, y=170)
        self.proximo = tk.Button(width='39', text='Gerar', font=('Arial','10'), command=self.relatorio)
        self.proximo.place(x=20, y=220)

    def relatorio(self):
        data = str(self.datas.get())
        df_solu = df_glpi[(df_glpi['Status'] == 'Fechado') | (df_glpi['Status'] == 'Solucionado')]
        #data = input('Qual ano e mês (ex: 2020-01): ')
        date = df_glpi['Data']#'Relatório da '+data+'.csv'
        df_solu[date >= data].groupby(['Area'])['Título'].value_counts().to_csv(filedialog.asksaveasfilename(defaultextension='.csv'),
                                                                                sep=';', encoding = 'utf-8-sig')

        messagebox.showinfo('Caixa de Messagem', 'Relatóro da '+data+' foi gerado')
        

janelaRaiz = tk.Tk()
janelaRaiz.title('UEMA GLPI')#Define o título da janela
janelaRaiz.geometry('500x300')
Tela(janelaRaiz)
janelaRaiz.configure(background = "#dde")
janelaRaiz.mainloop()